In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

In [3]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# 设置训练的设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# 定义预处理transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 加载CIFAR-10训练集，直接将官网 python 格式压缩包保存到运行目录，torch即可利用 CIFAR10 加载器加载数据
trainset = datasets.CIFAR10(root='./', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

# 加载CIFAR-10测试集
testset = datasets.CIFAR10(root='./', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

cuda
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# 初始化网络模型
net = SimpleCNN().to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
writer = SummaryWriter('cifar10')

In [6]:
# 训练网络
for epoch in range(30):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 获取输入数据
        inputs, labels = data[0].to(device), data[1].to(device)

        # 梯度归零
        optimizer.zero_grad()

        # 前向 + 反向 + 优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % (trainloader.batch_size / 4) == (trainloader.batch_size / 4) - 1:
            writer.add_scalar("Train/Loss", loss.item(), epoch*len(trainloader)+i)
        if i % trainloader.batch_size == trainloader.batch_size - 1:    # 按 batch_size 打印
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 125:.3f}')
            running_loss = 0.0

print('Finished Training')

[1, 64] loss: 1.179
[1, 128] loss: 1.178
[1, 192] loss: 1.177
[1, 256] loss: 1.176
[1, 320] loss: 1.174
[1, 384] loss: 1.172
[1, 448] loss: 1.169
[1, 512] loss: 1.163
[1, 576] loss: 1.154
[1, 640] loss: 1.138
[1, 704] loss: 1.112
[1, 768] loss: 1.087
[2, 64] loss: 1.064
[2, 128] loss: 1.044
[2, 192] loss: 1.029
[2, 256] loss: 1.016
[2, 320] loss: 0.997
[2, 384] loss: 0.988
[2, 448] loss: 0.976
[2, 512] loss: 0.975
[2, 576] loss: 0.950
[2, 640] loss: 0.945
[2, 704] loss: 0.929
[2, 768] loss: 0.912
[3, 64] loss: 0.903
[3, 128] loss: 0.893
[3, 192] loss: 0.878
[3, 256] loss: 0.875
[3, 320] loss: 0.843
[3, 384] loss: 0.843
[3, 448] loss: 0.836
[3, 512] loss: 0.838
[3, 576] loss: 0.817
[3, 640] loss: 0.818
[3, 704] loss: 0.803
[3, 768] loss: 0.800
[4, 64] loss: 0.791
[4, 128] loss: 0.795
[4, 192] loss: 0.781
[4, 256] loss: 0.770
[4, 320] loss: 0.770
[4, 384] loss: 0.780
[4, 448] loss: 0.761
[4, 512] loss: 0.772
[4, 576] loss: 0.756
[4, 640] loss: 0.740
[4, 704] loss: 0.764
[4, 768] loss: 0.

In [7]:
torch.save(net, 'cifar-10.pt')
print('cifar-10.pt saved')

cifar-10.pt saved


In [8]:
# 模型加载并训练
test_model = torch.load('cifar-10.pt')
test_model.train()

correct,total = 0,0
for j,data in enumerate(testloader):
    inputs,labels = data
    inputs,labels = inputs.to(device),labels.to(device)
    # 前向传播
    outputs = test_model(inputs)
    _, predicted = torch.max(outputs.data,1)
    total =total+labels.size(0)
    correct = correct +(predicted == labels).sum().item()
    # 准确率可视化
    if j % 20 == 0:
        writer.add_scalar("Train/Accuracy", 100.0*correct/total, j)

print('准确率：{:.4f}%'.format(100.0*correct/total))


准确率：74.0600%
